### Importar Dados

In [8]:
from avaliacao_swaps import *
from swap_trades import trade_params_df 
from processar_avaliacoes_repos_derivados import *
from queries import *
from dashboard_diario import process_files_for_patterns,repair_last_file,repair_and_save_excel
from dateutil.relativedelta import relativedelta
import os
reference_date = datetime.today()
reference_date_str = reference_date.strftime('%Y%m%d')
reference_date_str_2 = reference_date.strftime('%Y-%m-%d')
bbg_date= ql.TARGET().advance(ql.Date.todaysDate(), -1, ql.Days)
ql.Settings.instance().evaluationDate = bbg_date
curve_date = bbg_date.to_date()
curve_date_str = curve_date.strftime('%Y-%m-%d')
ctpy_valuation_date = curve_date.strftime('%Y%m%d')
db_paths = [r"\\igcp-fs\NCF-GER\Projetos 010-01-01\1_DESENVOLVIMENTO_PROJECTOS_INTERNOS\BLOOMBERG_DATABASE\swaps.db",
            r"\\igcp-fs\NCF-GER\Projetos 010-01-01\1_DESENVOLVIMENTO_PROJECTOS_INTERNOS\DASHBOARD_DIARIO\DB\DashboardDiario_database.db"]
data_dict = fetch_data(queries, db_paths, reference_date)
con_swaps = sqlite3.connect(db_paths[0])
con_dashboard = sqlite3.connect(db_paths[1])
df_3month = data_dict.get("query_3month")[0]
df_ois = data_dict.get("query_ois")[0]
df_fixings_estr = data_dict.get("query_estr_fixings")[0]
df_fixings_euri = data_dict.get("query_euribor_3m_fixings")[0]
zero_rate_ois = data_dict.get("query_historical_zero_rates_ois")[0]
zero_rate_3m = data_dict.get("query_historical_zero_rates_3m")[0]
zero_rate_ois["reference_date"] = pd.to_datetime(zero_rate_ois["reference_date"])
zero_rate_3m["reference_date"] = pd.to_datetime(zero_rate_3m["reference_date"])
zero_rate_ois = zero_rate_ois.sort_values("reference_date")
zero_rate_3m  = zero_rate_3m.sort_values("reference_date")
excel_dir_emails = r"\\igcp-fs\ncf-ger\Colateral 030-25-04\emails contrapartes derivados"
excel_dir_wss = excel_dir = "\\\\igcp.pt\\wss\\Import_Export\\Export\\PRD\\Dashboard_Export\\"
file_paths = {
    "MS": generate_file_path(excel_dir_emails, f"MS_Valuation_{ctpy_valuation_date}"),
    "CALY": generate_file_path(excel_dir_emails, f"CALY_Valuation_{ctpy_valuation_date}"),
    "BNPA": generate_file_path(excel_dir_emails, f"BNP_Valuation_{ctpy_valuation_date}"),
    }
file_patterns = [
        "NCF-DERIVADOS"
        ]
final_df=process_der_valuation_files(file_paths)
loaded_excels = process_files_for_patterns(reference_date, excel_dir_wss, file_patterns)
excel_derivados_data = loaded_excels[0]
df_derivados = excel_derivados_data.parse(sheet_name="derivatives", skiprows=1)
ois_curve, discount_handle, tenors_ois, quotes_ois = build_estr_curve(curve_date_str,df_ois)
euribor_curve, tenors_euribor, quotes_euribor = build_euribor_curve(curve_date_str,df_3month, df_ois)
sensibilidades = swaps_krr_data(bbg_date,reference_date,trade_params_df,df_ois,df_3month,df_fixings_estr,df_fixings_euri,ois_curve, discount_handle, tenors_ois, quotes_ois,euribor_curve, tenors_euribor, quotes_euribor, shift = 0.0001, payments=3)
VaR_parametrico_multi_fator = multi_factor_parametric_var(reference_date_str,curve_date_str, zero_rate_ois, zero_rate_3m, sensibilidades, horizon = "10D",alpha = 5/100,n_obs=3*30, choose_data = True)
pd.set_option('display.float_format', '{:.2f}'.format)
warnings.simplefilter(action='ignore', category=Warning)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
2025-09-04 00:00:00
Processing file: \\igcp.pt\wss\Import_Export\Export\PRD\Dashboard_Export\NCF-DERIVADOS_20250904_68.xlsx
File repaired successfully.
Repaired files for pattern 'NCF-DERIVADOS': ['\\\\igcp.pt\\wss\\Import_Export\\Export\\PRD\\Dashboard_Export\\NCF-DERIVADOS_20250904_68.xlsx']
VaR Paramétrico Multifator com dados históricos considerados até: 2025-06-06


### Alimentar tabela Monitorização de Limites de Derivados

In [6]:
atualizar_bd_monitorizacao_derivados(df_derivados, final_df, VaR_parametrico_multi_fator, reference_date_str_2, excel_serial_to_date,trade_params_df,con_dashboard,salvar = False )

,counterparty,reference_date,rating,maturidade_maxima,limite_maturidade,ocup_maturidade,limite_exposicao,exposicao,ocup_exposicao,nominal
0,BNPA PAR,2025-09-05,B,15.01,50.00,0.30,380000000.00,204409.42,0.00,125000000
1,CALY PAR,2025-09-05,B,15.01,50.00,0.30,380000000.00,202636.74,0.00,125000000
2,MSLN FFT,2025-09-05,B,15.01,50.00,0.30,380000000.00,1217534.93,0.00,1000000000


### Alimentar tabela Avaliação de Derivados

In [7]:
atualizar_avaliacao_der_bd(final_df, df_derivados,reference_date_str_2 ,con_dashboard,salvar = False)

,counterparty,reference_date,collateral_nominal,collateral_mv,igcp_derivatives_mv,mv_ctpy,mv_diff,movimento
0,MSLN FFT,2025-09-05,0.00,0.00,134017.34,11089.00,122928.34,0.00
1,CALY PAR,2025-09-05,0.00,0.00,56980.96,51620.05,5360.91,0.00
2,BNPA PAR,2025-09-05,0.00,0.00,74481.16,53324.99,21156.17,0.00
